In [1]:
from sqlalchemy.sql import text
import sqlalchemy as sa
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

engine = create_engine('mysql+pymysql://root@localhost:8081/airport-analytics?charset=utf8mb4')

# FOREIGN KEY tailnum_fk(tailnum) REFERENCES planes (tailnum), YES
# FOREIGN KEY dest_fk(dest) REFERENCES airports (faa), YES

In [2]:
weather = pd.read_csv('../static/weather.csv')
weather.fillna(np.nan, inplace = True)
weather.replace(r'\s+', np.nan, regex=True, inplace=True)

weather['time_hour'] = pd.to_datetime(weather['time_hour'], errors='coerce')

weather.to_sql('weather', con=engine, if_exists='append', index = False)

In [3]:
airlines = pd.read_csv('../static/airlines.csv')
airlines.fillna(np.nan, inplace = True)

airlines.to_sql('airlines', con=engine, if_exists='append', index = False)

In [4]:
airports = pd.read_csv('../static/airports.csv')
airports.fillna(np.nan, inplace = True)

airports.to_sql('airports', con=engine, if_exists='append', index = False)

In [5]:
planes = pd.read_csv('../static/planes.csv')
planes.fillna(np.nan, inplace = True)

planes['speed'].replace(r'\s+', np.nan, regex=True, inplace=True)
planes['year'].replace(r'\s+', np.nan, regex=True, inplace=True)

planes.to_sql('planes', con=engine, if_exists='append', index = False)

In [21]:
flights = pd.read_csv('../static/flights.csv')
flights.fillna(np.nan, inplace = True)

flights['time_hour'] = pd.to_datetime(flights['time_hour'], errors='coerce')
flights['arr_delay'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['air_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['arr_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['dep_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['dep_delay'].replace(r'\s+', np.nan, regex=True, inplace=True)

#--------------------------------------------------AIRPORTS FAA
airports_faa = []
for i in airports['faa']:
    airports_faa.append(i)
flights = flights[flights.dest.isin(airports_faa)]
#--------------------------------------------------PLANES TAILNUM
planes_tailnum = []
for i in planes['tailnum']:
    planes_tailnum.append(i)
#--------------------------------------------------
flights = flights[(flights.dest.isin(airports_faa)) & (flights.tailnum.isin(planes_tailnum))]

index = flights.index
rows_count = len(index)
print(rows_count)


277977


In [22]:
flights.to_sql('flights', con=engine, if_exists='append', index = False)